In [1]:
import streamlit as st
from streamlit_jupyter import StreamlitPatcher, tqdm

from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
import langchain_community.vectorstores.faiss as FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

from dotenv import load_dotenv

In [ ]:
load_dotenv()
genai.configure(api_key = "")

In [3]:
model = ChatGoogleGenerativeAI(model = "gemini-2.0-flash")

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [9]:
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

In [10]:
def get_test_chunks(text):

    text_splitter = RecursiveCharacterTextSplitter(chuck_size = 10000, chunk_overlap = 1000)
    chunks = text_splitter.split_text(text)
    return chunks

In [11]:
def grt_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    vecor_store = FAISS.from_texts(text_chunks, embedding = embeddings)
    vecor_store.save_local("faiss_index")

In [ ]:
def get_conversational_chain():
    prompt_template = """
    Answer teh question as detailed asn possible from the provided context, make sure to provide all teh details, if the answer is not in provided context just say, "answer is not available in the context", don't provide wrong answers \n\n
    context: \n{context}?\n
    question: \n{question}\n

    Answer:

    """

    model = ChatGoogleGenerativeAI(model = "gemini-pro", temperature=0.3)

    prompt = PromptTemplate(template=prompt_template, input_variables=["context","question"])
    chain = load_qa_chain(model,chain_type="stuff", prompt = prompt)
    return chain

In [13]:
def user_input(user_question):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    
    new_db = FAISS.load_local("faiss_index", embeddings)
    docs = new_db.similarity_search(user_question)

    chain = get_conversational_chain()

    
    response = chain(
        {"input_documents":docs, "question": user_question}
        , return_only_outputs=True)

    print(response)
    st.write("Reply: ", response["output_text"])

In [ ]:
st.set_page_config("Chat PDF")
st.header("Chat with PDF using Gemini💁")
user_question = st.text_input("Ask a Question from the PDF Files")

if user_question:
    user_input(user_question)

with st.sidebar:
    st.title("Menu:")
    pdf_docs = st.file_uploader("Upload your PDF Files and Click on the Submit & Process Button", accept_multiple_files=True)
    if st.button("Submit & Process"):
        with st.spinner("Processing..."):
            raw_text = get_pdf_text(pdf_docs)
            text_chunks = get_text_chunks(raw_text)
            get_vector_store(text_chunks)
            st.success("Done")

2025-03-13 11:46:35.707 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 11:46:35.708 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 11:46:35.725 
  command:

    streamlit run /Users/muralikoripalli/Desktop/reume parser/project_env/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-03-13 11:46:35.725 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 11:46:35.725 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 11:46:35.726 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-13 11:46:35.726 Thread 'MainThread': missing ScriptRunCont